<a href="https://colab.research.google.com/github/jiyeonjin/0624_new/blob/main/0728.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ZIP 파일을 코랩으로 복사
!cp "/content/drive/MyDrive/Colab Notebooks/dataset.zip" /content/

# 압축 해제
!unzip -o /content/dataset.zip -d /content/

# 압축 해제 확인
!ls -la /content/

!find /content -name best.pt

In [ ]:
# 1. ultralytics 설치
!pip install ultralytics

# 2. 설치 확인 후 다시 실행
from ultralytics import YOLO
import glob
import os

print("✅ ultralytics 설치 완료!")

# 이미 학습된 모델 사용
model = YOLO('/content/dataset/best.pt')

# YouTube 영상 다운로드
!pip install yt-dlp
!yt-dlp -f 'best[height<=720]' -o '/content/test_video.%(ext)s' 'https://www.youtube.com/watch?v=AxLmroTo3rQ'

# 다운로드된 파일 찾기 (확장자가 다를 수 있음)
video_files = glob.glob('/content/test_video.*')
if video_files:
    video_path = video_files[0]
    print(f"📹 다운로드된 영상: {video_path}")

    # 추론 실행
    results = model(video_path)

    # 결과 표시 (영상의 경우 첫 번째 프레임만)
    if results:
        results[0].show()
else:
    print("❌ 영상 다운로드 실패")

# 기존 검증 데이터로 성능 측정
print("\n📊 모델 성능 평가:")
metrics = model.val(data='/content/dataset/dataset.yaml')
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

print("\n✅ 모든 작업 완료!")

In [ ]:
# 1. dataset.yaml 파일 내용 확인
print("📋 dataset.yaml 파일 내용:")
with open('/content/dataset/dataset.yaml', 'r') as f:
    yaml_content = f.read()
    print(yaml_content)

In [ ]:
!pip install ultralytics yt-dlp

from ultralytics import YOLO
import glob

# yaml 수정 (핵심 문제 해결)
yaml_fix = '''path: /content/dataset
train: train/images
val: valid/images
names:
  0: lane
  1: traffic_sign
nc: 2'''

with open('/content/dataset/dataset_fixed.yaml', 'w') as f:
    f.write(yaml_fix)

# 모델 로드 & 영상 다운로드 & 추론
model = YOLO('/content/dataset/best.pt')
!yt-dlp -f 'best[height<=720]' -o '/content/test_video.%(ext)s' 'https://www.youtube.com/watch?v=AxLmroTo3rQ'

video_path = glob.glob('/content/test_video.*')[0]
results = model(video_path)
results[0].show()

# 성능 평가
metrics = model.val(data='/content/dataset/dataset_fixed.yaml')
print(f"mAP50: {metrics.box.map50:.4f}")

In [ ]:
!pip install ultralytics yt-dlp

from ultralytics import YOLO
import glob
import cv2
import numpy as np
from IPython.display import Video
import shutil

# yaml 수정 (핵심 문제 해결)
yaml_fix = '''path: /content/dataset
train: train/images
val: valid/images
names:
  0: lane
  1: traffic_sign
nc: 2'''

with open('/content/dataset/dataset_fixed.yaml', 'w') as f:
    f.write(yaml_fix)

# 모델들 로드
print("🤖 모델 로드 중...")
base_model = YOLO('yolo11n.pt')  # 기본 YOLO (80개 클래스)
custom_model = YOLO('/content/dataset/best.pt')  # 커스텀 YOLO (2개 클래스)

print(f"기본 모델 클래스 수: {len(base_model.names)}")
print(f"커스텀 모델 클래스 수: {len(custom_model.names)}")

# 영상 다운로드
print("📥 YouTube 영상 다운로드 중...")
!yt-dlp -f 'best[height<=720]' -o '/content/test_video.%(ext)s' 'https://www.youtube.com/watch?v=AxLmroTo3rQ'

video_path = glob.glob('/content/test_video.*')[0]
print(f"✅ 다운로드 완료: {video_path}")

# 결합된 추론 함수
def combined_inference(video_path, output_path='/content/combined_result.mp4'):
    """기본 YOLO + 커스텀 YOLO 결과를 결합하여 영상 생성"""

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 출력 영상 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"🎬 영상 처리 중... (총 {total_frames} 프레임)")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 기본 YOLO 추론 (차량, 사람, 신호등 등)
        base_results = base_model(frame, verbose=False)

        # 커스텀 YOLO 추론 (차선, 교통표지판)
        custom_results = custom_model(frame, verbose=False)

        # 결과 시각화
        annotated_frame = frame.copy()

        # 기본 YOLO 결과 그리기 (파란색)
        if base_results[0].boxes is not None:
            for box in base_results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])

                if conf > 0.3:  # 신뢰도 30% 이상만
                    label = f"{base_model.names[cls]} {conf:.2f}"
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # 파란색
                    cv2.putText(annotated_frame, label, (x1, y1-10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # 커스텀 YOLO 결과 그리기 (빨간색)
        if custom_results[0].boxes is not None:
            for box in custom_results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])

                if conf > 0.3:  # 신뢰도 30% 이상만
                    label = f"{custom_model.names[cls]} {conf:.2f}"
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # 빨간색
                    cv2.putText(annotated_frame, label, (x1, y1-10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        out.write(annotated_frame)
        frame_count += 1

        if frame_count % 30 == 0:  # 30프레임마다 진행상황 출력
            print(f"   처리 중... {frame_count}/{total_frames} ({frame_count/total_frames*100:.1f}%)")

    cap.release()
    out.release()
    print(f"✅ 결합 결과 영상 저장: {output_path}")

# 개별 추론 결과
print("\n🔍 개별 모델 추론 결과:")

# 커스텀 모델만으로 추론
print("1️⃣ 커스텀 모델 (차선, 교통표지판만):")
custom_results = custom_model(video_path, save=True, project='/content', name='custom_only')

# 기본 모델만으로 추론
print("2️⃣ 기본 모델 (일반 객체들):")
base_results = base_model(video_path, save=True, project='/content', name='base_only')

# 결합된 추론
print("3️⃣ 결합 모델 (모든 객체):")
combined_inference(video_path, '/content/combined_result.mp4')

# 결과 영상들을 표준 이름으로 복사
result_files = {
    'custom_result.mp4': glob.glob('/content/custom_only/*.avi') + glob.glob('/content/custom_only/*.mp4'),
    'base_result.mp4': glob.glob('/content/base_only/*.avi') + glob.glob('/content/base_only/*.mp4'),
    'final_combined_result.mp4': ['/content/combined_result.mp4']
}

print("\n📁 결과 파일들:")
for name, files in result_files.items():
    if files and files[0]:
        shutil.copy(files[0], f'/content/{name}')
        print(f"✅ {name} 생성 완료")

# 성능 평가 (커스텀 모델)
print("\n📊 커스텀 모델 성능 평가:")
metrics = custom_model.val(data='/content/dataset/dataset.yaml')
print(f"mAP50: {metrics.box.map50:.4f}")

# 최종 결과 영상 재생
print("\n🎬 최종 결합 결과 영상:")
Video('/content/final_combined_result.mp4', width=800)

print("\n🎯 결과 요약:")
print("🔵 파란색 박스: 기본 YOLO (차량, 사람, 신호등, 표지판 등)")
print("🔴 빨간색 박스: 커스텀 YOLO (차선, 교통표지판)")
print("\n💾 다운로드 가능한 파일들:")
print("- custom_result.mp4: 커스텀 모델만")
print("- base_result.mp4: 기본 모델만")
print("- final_combined_result.mp4: 모든 객체 탐지")